In [20]:
import gradio as gr
import os
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import asyncio
from dotenv import load_dotenv
import nest_asyncio
from groq import Groq
import pdfplumber
import pytesseract
from PIL import Image
import io
import fitz  # PyMuPDF
import tempfile
import re
import time
import shutil

# Load environment variables
load_dotenv()
nest_asyncio.apply()

# Initialize Sentence Transformer for embeddings
embedding_model = SentenceTransformer('multi-qa-MiniLM-L6-cos-v1')

# Initialize FAISS index
dimension = 384
faiss_index = faiss.IndexFlatL2(dimension)

# Memory to store previously generated questions
question_memory = set()

# Store chunk to document mapping
chunk_doc_mapping = []

# Initialize Groq GPT-4 client
api_key = ("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")
if not api_key:
    raise ValueError("GROQ_API_KEY is missing. Set it in the .env file.")
client = Groq(api_key=api_key)

# Function to extract text from PDFs with OCR capability
def extract_text_from_pdfs(pdf_files):
    start_time = time.time()
    all_texts = []  # Store text with document ID
    processed_files = 0

    for doc_id, pdf_file in enumerate(pdf_files):
        try:
            # Get the file path directly - Gradio provides the file path
            pdf_path = pdf_file.name

            # First try with pdfplumber for text-based PDFs
            text_from_plumber = ""
            with pdfplumber.open(pdf_path) as pdf:
                for page in pdf.pages:
                    extracted = page.extract_text()
                    if extracted:
                        text_from_plumber += extracted + " "

            # If pdfplumber extracted substantial text, use it
            if len(text_from_plumber.strip()) > 100:  # Arbitrary threshold
                all_texts.append((text_from_plumber, doc_id, os.path.basename(pdf_path)))
            else:
                # Fall back to OCR with PyMuPDF
                doc = fitz.open(pdf_path)
                ocr_text = ""
                for page_num in range(len(doc)):
                    page = doc.load_page(page_num)
                    pix = page.get_pixmap(matrix=fitz.Matrix(300/72, 300/72))
                    img = Image.open(io.BytesIO(pix.tobytes("png")))
                    ocr_text += pytesseract.image_to_string(img) + "\n"
                doc.close()
                all_texts.append((ocr_text, doc_id, os.path.basename(pdf_path)))

            processed_files += 1

        except Exception as e:
            return f"Error extracting text from {os.path.basename(pdf_file.name)}: {str(e)}"

    end_time = time.time()
    processing_time = end_time - start_time

    return all_texts, processed_files, processing_time

# Split text into chunks with document tracking
def split_text_into_chunks(text, doc_id, doc_name, chunk_size=750):
    global chunk_doc_mapping
    sentences = re.split(r'(?<=[.!?]) +', text)
    chunks, chunk = [], ""
    chunk_ids = []

    for sentence in sentences:
        if len(chunk) + len(sentence) <= chunk_size:
            chunk += " " + sentence
        else:
            chunks.append(chunk.strip())
            chunk_ids.append((doc_id, doc_name))
            chunk = sentence

    if chunk:
        chunks.append(chunk.strip())
        chunk_ids.append((doc_id, doc_name))

    # Update global mapping
    chunk_doc_mapping.extend(chunk_ids)
    return chunks

# Store embeddings in FAISS asynchronously with batching
async def store_embeddings_in_faiss(text_chunks):
    batch_size = 100
    for i in range(0, len(text_chunks), batch_size):
        batch = text_chunks[i:i + batch_size]
        embeddings = embedding_model.encode(batch)
        faiss_index.add(np.array(embeddings).astype('float32'))
        # Small pause to prevent resource overload
        if i + batch_size < len(text_chunks):
            await asyncio.sleep(0.1)

# Retrieve relevant text chunks using FAISS with improved diversity
def retrieve_relevant_text(query, num_docs, min_chunks_per_doc=2, max_total_chunks=20):
    # Calculate a reasonable number of chunks to retrieve
    base_chunks = min_chunks_per_doc * num_docs
    total_chunks = min(base_chunks + 5, max_total_chunks)  # Add some extras for relevance

    # Get more chunks than we need to ensure document coverage
    query_embedding = embedding_model.encode([query])
    distances, indices = faiss_index.search(np.array(query_embedding).astype('float32'), total_chunks * 2)
    retrieved_indices = indices[0]

    # Ensure document diversity in the selection
    selected_indices = select_diverse_chunks(retrieved_indices, num_docs, min_chunks_per_doc)

    return selected_indices[:total_chunks]  # Limit to reasonable number

# Select diverse chunks to ensure all documents are represented
def select_diverse_chunks(retrieved_indices, num_docs, min_per_doc=1):
    global chunk_doc_mapping

    # Initialize tracking
    selected_chunks = []
    doc_counts = {i: 0 for i in range(num_docs)}
    doc_chunks = {i: [] for i in range(num_docs)}

    # First sort each chunk into its document bucket
    for idx in retrieved_indices:
        if idx < len(chunk_doc_mapping):
            doc_id = chunk_doc_mapping[idx][0]
            if doc_id < num_docs:  # Ensure valid doc_id
                doc_chunks[doc_id].append(idx)

    # First ensure minimum representation from each document
    for doc_id in range(num_docs):
        chunks = doc_chunks.get(doc_id, [])
        for i, chunk_idx in enumerate(chunks):
            if i < min_per_doc:  # Take at least min_per_doc from each document
                selected_chunks.append(chunk_idx)
                doc_counts[doc_id] += 1

    # Then add remaining chunks in order of relevance (they were already sorted by distance)
    for idx in retrieved_indices:
        if idx < len(chunk_doc_mapping) and idx not in selected_chunks:
            selected_chunks.append(idx)

    return selected_chunks

# Evaluate question quality
def evaluate_question(question):
    if question in question_memory:
        return False
    question_memory.add(question)
    return True

# Function to calculate mark allocation based on difficulty and question type
def calculate_mark_allocation(difficulty, num_mcq, num_subjective, total_marks=100):
    # Adjust base mark values according to difficulty
    if difficulty == "Easy":
        mcq_base_marks = 1
        subjective_base_marks_min = 5
        subjective_base_marks_max = 10
    elif difficulty == "Medium":
        mcq_base_marks = 2
        subjective_base_marks_min = 8
        subjective_base_marks_max = 15
    else:  # Hard
        mcq_base_marks = 3
        subjective_base_marks_min = 10
        subjective_base_marks_max = 20

    # Calculate how many marks to allocate to MCQs and subjective questions
    total_mcq_marks = mcq_base_marks * num_mcq

    # Remaining marks go to subjective questions with variability
    remaining_marks = total_marks - total_mcq_marks

    # If there are subjective questions, distribute marks among them
    if num_subjective > 0:
        # Create a distribution of marks for subjective questions
        subjective_marks = []

        # For complex distribution, assign variable marks based on complexity
        # First, ensure minimum marks for each question
        min_marks_per_subjective = subjective_base_marks_min
        total_min_subjective = min_marks_per_subjective * num_subjective

        # If minimum exceeds remaining, adjust proportionally
        if total_min_subjective > remaining_marks:
            min_marks_per_subjective = remaining_marks // num_subjective
            subjective_marks = [min_marks_per_subjective] * num_subjective
        else:
            # Distribute remaining marks with some questions worth more
            extra_marks = remaining_marks - total_min_subjective
            subjective_marks = [min_marks_per_subjective] * num_subjective

            # Distribute extra marks to create variability
            import random
            # Set a seed for reproducibility
            random.seed(42)

            # Distribute extra marks with bias toward later questions
            weights = [i+1 for i in range(num_subjective)]  # Higher weight for later questions
            total_weight = sum(weights)

            for i in range(num_subjective):
                # Calculate proportional share of extra marks
                extra_for_this = int((weights[i] / total_weight) * extra_marks)
                subjective_marks[i] += extra_for_this

            # Assign any remaining marks to the last question
            remaining = remaining_marks - sum(subjective_marks)
            if remaining > 0:
                subjective_marks[-1] += remaining

        # Ensure no question exceeds maximum marks for difficulty
        for i in range(len(subjective_marks)):
            if subjective_marks[i] > subjective_base_marks_max:
                excess = subjective_marks[i] - subjective_base_marks_max
                subjective_marks[i] = subjective_base_marks_max

                # Redistribute excess marks to other questions below max
                for j in range(len(subjective_marks)):
                    if j != i and subjective_marks[j] < subjective_base_marks_max and excess > 0:
                        room = subjective_base_marks_max - subjective_marks[j]
                        add = min(room, excess)
                        subjective_marks[j] += add
                        excess -= add
                        if excess == 0:
                            break
    else:
        subjective_marks = []

    return mcq_base_marks, subjective_marks

# Generate questions using Groq
def generate_questions(relevant_chunks, text_chunks, subject, num_questions, num_mcq, num_subjective, difficulty, doc_count, total_marks=100, question_types=None):
    questions = []

    # Get mark allocation
    mcq_marks, subjective_marks = calculate_mark_allocation(difficulty, num_mcq, num_subjective, total_marks)

    # Get the corresponding text chunks and document info
    doc_representation = {}
    relevant_text = ""

    for idx in relevant_chunks:
        if idx < len(text_chunks) and idx < len(chunk_doc_mapping):
            chunk_text = text_chunks[idx]
            doc_id, doc_name = chunk_doc_mapping[idx]

            # Track which documents are represented
            if doc_id not in doc_representation:
                doc_representation[doc_id] = 1
            else:
                doc_representation[doc_id] += 1

            # Add document source to the text
            relevant_text += f"\n--- From document: {doc_name} ---\n{chunk_text}\n"

    # Define question types if not provided
    if not question_types:
        question_types = {
            "bloom": True,
            "competency": True,
            "case_study": False,
            "fill_blanks": False
        }

    # Format the mark information for the prompt
    mcq_mark_info = f"Each MCQ question is worth {mcq_marks} marks."

    subjective_mark_info = ""
    if subjective_marks:
        subjective_mark_info = "Subjective questions have the following mark allocation:\n"
        for i, marks in enumerate(subjective_marks):
            subjective_mark_info += f"- Question {i+1}: {marks} marks\n"

    # Build a prompt that emphasizes using ALL documents with taxonomies but removes visible tags
    prompt = f"""
    You are an expert in {subject}. Generate {num_questions} unique and high-quality questions with marks assigned to each question.

    QUESTION STRUCTURE:
    - {num_mcq} questions should be in "Section - 1 MCQ". These should be multiple-choice (MCQ) questions with 4 options each.
    - {num_subjective} questions should be in "Section - 2 Subjective". These should be subjective/essay questions.
    - Ensure the difficulty level is {difficulty}.

    MARK ALLOCATION:
    The total exam is worth {total_marks} marks.
    {mcq_mark_info}
    {subjective_mark_info}

    FORMATTING REQUIREMENTS:
    - Include the mark value in parentheses after each question number, like: "Question 1 ({mcq_marks} marks)"
    - For subjective questions, show the specific mark value like: "Question 1 ({subjective_marks[0] if subjective_marks else 0} marks)"

    - At the start of the question paper, include a line showing "Total Marks: {total_marks}"

    IMPORTANT METADATA REQUIREMENTS:
    """

    if question_types.get("bloom", False):
        prompt += """
    BLOOM'S TAXONOMY (INTERNAL USE ONLY):
    Internally categorize questions across different cognitive levels of Bloom's Taxonomy:
    1. Remember (Knowledge): Questions testing recall of facts, terms, basic concepts
    2. Understand (Comprehension): Questions testing understanding of meaning
    3. Apply: Questions testing use of information in new situations
    4. Analyze: Questions testing breaking down of information and exploring relationships
    5. Evaluate: Questions testing justification of decisions or actions
    6. Create: Questions testing generation of new ideas or products

    IMPORTANT: DO NOT add any visible Bloom's Taxonomy labels or tags to the questions in your final output.
    """

    if question_types.get("competency", False):
        prompt += """
    COMPETENCY-BASED CATEGORIES (INTERNAL USE ONLY):
    Internally categorize questions that assess different competencies:
    - Conceptual Understanding: Tests grasp of fundamental concepts
    - Critical Thinking: Tests ability to evaluate information and make judgments
    - Problem-Solving: Tests ability to apply knowledge to solve real-world problems
    - Communication: Tests ability to articulate ideas effectively
    - Research Skills: Tests ability to gather and interpret information

    IMPORTANT: DO NOT add any visible competency category labels or tags to the questions in your final output.
    """

    if question_types.get("case_study", False):
        prompt += """
    CASE STUDY:
    Include at least one case study question that presents a scenario and asks multiple related questions about it.
    DO NOT explicitly label it as a case study in the output.
    """

    if question_types.get("fill_blanks", False):
        prompt += """
    FILL IN THE BLANKS:
    Include fill-in-the-blank questions where appropriate.
    Format them clearly with underscores or blank spaces to indicate where answers should be filled in.
    DO NOT explicitly label them as fill-in-the-blank questions.
    """

    prompt += f"""
    IMPORTANT OUTPUT REQUIREMENTS:
    - DO NOT include any taxonomy labels, tags, or categorization markers in the final questions.
    - The questions should appear clean without any [Bloom level], {{competency}}, or other classification tags.
    - Create questions that cover content from ALL {doc_count} documents provided.
    - Try to distribute questions evenly across all the source documents.
    - Ensure questions are non-repetitive and cover different concepts.
    - Provide a mix of question formats while maintaining the MCQ and subjective counts.
    - In the final section titled "ANSWER KEY", provide the answers for all questions and for each answer, specify which document it's based on (e.g., [Source: document1.pdf]).
    - At the very end, include a "MARK DISTRIBUTION" section that shows the total marks and breakdown by section.

    Here's the reference text with document sources:
    {relevant_text}
    """

    try:
        response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": prompt}],
            max_tokens=2500  # Increased to allow for more comprehensive responses
        )
        generated_text = response.choices[0].message.content.strip()

        # Post-process to remove any taxonomy tags that might have been included despite instructions
        processed_text = re.sub(r'\[(Remember|Understand|Apply|Analyze|Evaluate|Create)\]', '', generated_text)
        processed_text = re.sub(r'\{(Conceptual Understanding|Critical Thinking|Problem-Solving|Communication|Research Skills)\}', '', processed_text)
        processed_text = re.sub(r'\[CASE STUDY\]', '', processed_text)
        processed_text = re.sub(r'\[FILL IN THE BLANKS\]', '', processed_text)

        # Clean up any double spaces resulting from tag removal
        processed_text = re.sub(r' +', ' ', processed_text)

        questions = processed_text.split("\n\n")  # Splitting based on paragraphs

        # Ensure uniqueness
        questions = [q for q in questions if evaluate_question(q)]

    except Exception as e:
        return f"Error generating questions: {str(e)}"

    return "\n\n".join(questions)

# Function to add missing marks from the original implementation
def generate_question_bank(pdf_files, subject, num_questions, num_mcq, num_subjective, difficulty, question_types, total_marks=100):
    global faiss_index, chunk_doc_mapping

    # Reset global variables for new processing
    faiss_index = faiss.IndexFlatL2(dimension)
    chunk_doc_mapping = []
    question_memory.clear()

    try:
        # Extract text from PDFs
        result = extract_text_from_pdfs(pdf_files)
        if isinstance(result, str):  # Error occurred
            return result

        all_texts, processed_files, processing_time = result

        # Process the texts into chunks
        all_chunks = []
        for text, doc_id, doc_name in all_texts:
            chunks = split_text_into_chunks(text, doc_id, doc_name)
            all_chunks.extend(chunks)

        # Store embeddings asynchronously
        asyncio.run(store_embeddings_in_faiss(all_chunks))

        # Create a query based on subject and difficulty
        query = f"{subject} {difficulty} level questions"

        # Retrieve relevant chunks
        relevant_indices = retrieve_relevant_text(query, len(all_texts))

        # Generate questions with mark allocation
        questions = generate_questions(relevant_indices, all_chunks, subject,
                                     num_questions, num_mcq, num_subjective,
                                     difficulty, len(all_texts), total_marks, question_types)

        return questions

    except Exception as e:
        return f"Error in question bank generation: {str(e)}"

# Process function with metadata for internal tracking but clean output
def process_with_status(pdf_files, subject, num_questions, num_mcq, num_subjective, difficulty,
                       bloom_taxonomy, competency_based, case_study, fill_blanks, total_marks=100):
    status_value = f"⏳ Processing {len(pdf_files)} PDFs..."
    yield status_value, ""

    question_types = {
        "bloom": bloom_taxonomy,
        "competency": competency_based,
        "case_study": case_study,
        "fill_blanks": fill_blanks
    }

    result = generate_question_bank(pdf_files, subject, num_questions, num_mcq, num_subjective,
                                   difficulty, question_types, total_marks)

    # Additional cleanup to ensure no tags are visible
    clean_result = re.sub(r'\[(Remember|Understand|Apply|Analyze|Evaluate|Create)\]\s*', '', result)
    clean_result = re.sub(r'\{(Conceptual Understanding|Critical Thinking|Problem-Solving|Communication|Research Skills)\}\s*', '', clean_result)
    clean_result = re.sub(r'\[CASE STUDY\]\s*', '', clean_result)
    clean_result = re.sub(r'\[FILL IN THE BLANKS\]\s*', '', clean_result)
    clean_result = re.sub(r' +', ' ', clean_result)

    status_value = "✅ Enhanced question bank generation complete!"
    yield status_value, clean_result

# Enhanced Gradio UI with a modern, aesthetic design including total marks
with gr.Blocks(css="""
    body { background: linear-gradient(to right, #4A00E0, #8E2DE2); color: white; }
    .gradio-container { background: #ffffff; border-radius: 12px; box-shadow: 0 4px 15px rgba(0, 0, 0, 0.2); padding: 25px; }
    .gradio-button { background: linear-gradient(45deg, #ff416c, #ff4b2b); color: white; font-weight: bold; border-radius: 8px; border: none; padding: 12px 20px; transition: 0.3s; }
    .gradio-button:hover { transform: scale(1.05); }
    .gr-textbox { border: 2px solid #6a11cb; border-radius: 8px; padding: 8px; }
    .progress-text { color: #ff416c; font-weight: bold; }
    h1 { text-align: center; font-size: 2rem; margin-bottom: 10px; }
    h3 { color: #6a11cb; text-align: center; margin-bottom: 20px; }
    .category-title { font-weight: bold; margin-top: 10px; color: #6a11cb; }
    """) as demo:

    gr.Markdown("""
    # ✨ Enhanced Question Bank Generator
    **AI-Powered with Advanced Learning Frameworks**
    """)

    with gr.Row():
        pdf_files = gr.Files(label="📄 Upload PDFs", file_types=[".pdf"])

    with gr.Row():
        with gr.Column():
            subject = gr.Textbox(label="📚 Subject", placeholder="Enter subject name", elem_classes=["gr-textbox"])
            difficulty = gr.Dropdown(label="🎯 Difficulty Level", choices=["Easy", "Medium", "Hard"], value="Medium")
            total_marks = gr.Slider(label="📊 Total Marks", minimum=50, maximum=200, value=100, step=10)

        with gr.Column():
            num_questions = gr.Slider(label="❓ Total Questions", minimum=1, maximum=20, value=10, step=1)
            num_mcq = gr.Slider(label="🔘 MCQs", minimum=0, maximum=15, value=5, step=1)
            num_subjective = gr.Slider(label="✍️ Subjective", minimum=0, maximum=15, value=5, step=1)

    with gr.Row():
        gr.Markdown("""### 🧠 Question Categories (Internal Classification Only)""")

    with gr.Row():
        with gr.Column():
            bloom_taxonomy = gr.Checkbox(label="Use Bloom's Taxonomy (internal only)", value=True)
            bloom_info = gr.Markdown("""
            <div class="category-title">Questions will be created using these cognitive levels:</div>
            - Remember: Knowledge recall
            - Understand: Comprehension
            - Apply: Use in new situations
            - Analyze: Break down information
            - Evaluate: Justify decisions
            - Create: Generate new ideas
            """)

        with gr.Column():
            competency_based = gr.Checkbox(label="Use Competency-Based Categories (internal only)", value=True)
            competency_info = gr.Markdown("""
            <div class="category-title">Questions will be created using these competencies:</div>
            - Conceptual Understanding
            - Critical Thinking
            - Problem-Solving
            - Communication
            - Research Skills
            """)

    with gr.Row():
        with gr.Column():
            case_study = gr.Checkbox(label="Include Case Study Questions", value=False)
        with gr.Column():
            fill_blanks = gr.Checkbox(label="Include Fill-in-the-Blanks", value=False)

    status = gr.Textbox(label="🚀 Status", value="Ready to process", interactive=False)
    generate_button = gr.Button("⚡ Generate Question Bank with Marks", elem_classes=["gradio-button"])
    output = gr.Textbox(label="📜 Generated Question Bank", lines=25, interactive=False)

    generate_button.click(
        process_with_status,
        inputs=[pdf_files, subject, num_questions, num_mcq, num_subjective, difficulty,
               bloom_taxonomy, competency_based, case_study, fill_blanks, total_marks],
        outputs=[status, output]
    )

# Add an exception handler to catch any errors during launch
try:
    demo.launch(share=True)
except Exception as e:
    print(f"Error launching Gradio app: {str(e)}")

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://57c9440556f9ca008e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Error launching Gradio app: module 'gradio' has no attribute 'blocks'


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://c5b2bad6ff58fdffb6.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Error launching Gradio app: module 'gradio' has no attribute 'blocks'
